# Orq Knowledge Base SDK Usage Examples

This notebook provides practical examples of using the Orq SDK to work with Knowledge Bases. The examples cover all the key operations from the OpenAPI schema including:

- Listing knowledge bases
- Creating a knowledge base
- Retrieving a knowledge base
- Updating a knowledge base
- Deleting a knowledge base
- Searching a knowledge base
- Working with datasources
- Working with chunks

Let's start by setting up the SDK.

### Setup

Before trying out the code set up the virtual environment and the environment variable. 

On the terminal go to the project directory, activate the virtual environment and install Orq sdk:

**pip install orq-ai-sdk**

Next set up the API key:

In [ ]:
import os

os.environ["ORQ_API_KEY"] = "" # Your Api key here

In [ ]:
from orq_ai_sdk import Orq
import os

orq = Orq(server_url="", api_key=os.getenv("ORQ_API_KEY"))

In [ ]:
import json

# Function to pretty print JSON responses
def pretty_print(obj):
    print(json.dumps(obj, indent=2))
    
def print_chunk(chunk):
    print(f"  ID       : {chunk.id}")
    print(f"  Status   : {chunk.status}")
    print(f"  Enabled  : {chunk.enabled}")
    print(f"  Created  : {chunk.created}")
    print(f"  Updated  : {chunk.updated}")
    print(f"  Metadata : {chunk.metadata}")
    print(f"  Text     : {chunk.text}")
    print("-" * 60)

## 1. List all knowledge bases

Returns a list of your knowledge bases. The knowledge bases are returned sorted by creation date, with the most recent knowledge bases appearing first

In [ ]:
# List all knowledge bases (default limit of 10)
kb_list = orq.knowledge.list()

if len(kb_list.data) > 0:
   print("Your knowledge bases:")
   for kb in kb_list.data:
      print(kb.id)
else:
   print("No knowledge added yet")

print("")
# With pagination parameters
kb_list_paginated = orq.knowledge.list(
   limit=10
)
pretty_print(json.loads(kb_list_paginated.model_dump_json()))

## 2. Create a knowledge

Endpoint: `POST /v2/knowledge`

In [ ]:
# Create a new knowledge base with vector search retrieval settings
new_kb = orq.knowledge.create(
    key="Testing_KB",  # Unique identifier for the knowledge base
    embedding_model="cohere/embed-multilingual-v3.0",  # Embeddings model to use
    path="Default",  # Path in the project structure
    description="Knowledge base for product documentation and FAQs",  # Optional description
    retrieval_settings={
        "type": "vector_search",
        "top_k": 5,  # Return top 5 most relevant chunks
        "threshold": 0.7,  # Minimum similarity score (0-1)
        "rerank_config": {
            "enabled": True,
            "provider": "cohere",
            "rerank_model": "cohere/embed-multilingual-v3.0",
            "model_type": "rerank"
        }
    }
)

# Store the knowledge base ID for later use
kb_id = new_kb.id

print(f"Created knowledge base with ID: {kb_id}")

pretty_print(json.loads(new_kb.model_dump_json()))

## 3. Retrieves a knowledge base

Retrieve a knowledge base with the settings.

In [ ]:
# Example for `retrieve`
kb_id = "" # enter the knowledge base id here

# Retrieve a specific knowledge base
kb_details = orq.knowledge.retrieve(knowledge_id=kb_id)

pretty_print(json.loads(kb_details.model_dump_json()))

## 4. Updates a knowledge

Endpoint: `PATCH /v2/knowledge/{knowledge_id}`

In [ ]:
# Update the knowledge base settings

kb_id = "" # enter the knowledge base id here

updated_kb = orq.knowledge.update(
    knowledge_id=kb_id,
    description="Updated product documentation knowledge base",
    retrieval_settings={
        "type": "hybrid_search",  # Change to hybrid search
        "top_k": 8,  # Increase top_k to 8
        "threshold": 0.6  # Lower threshold to 0.6
    }
)
pretty_print(json.loads(updated_kb.model_dump_json()))

## 5. Deletes a knowledge

Deletes a knowledge base. Deleting a knowledge base will delete all the datasources and chunks associated with it.

In [ ]:
kb_id = "" # enter the knowledge base id here

# list kb before deletion (default limit of 10)
kb_list = orq.knowledge.list()

if len(kb_list.data) > 0:
   print("Your knowledge bases before:")
   print("knowledge base count:", len(kb_list.data))
   for kb in kb_list.data:
      print(kb.id)
else:
   print("No knowledge added yet")

# Example for `delete`
response = orq.knowledge.delete(
    knowledge_id = kb_id)

print("")

# list kb after deletion (default limit of 10)
kb_list = orq.knowledge.list()

if len(kb_list.data) > 0:
   print("Your knowledge bases after:")
   print("knowledge base counnt:", len(kb_list.data))
   for kb in kb_list.data:
      print(kb.id)
else:
   print("No knowledge added yet")

## 6. Retrieves the documents used for retrieval

Endpoint: `POST /v2/knowledge/{knowledge_id}/search`

In [ ]:
kb_id = "" # enter the knowledge base id here

# Search for documents in the knowledge base
search_results = orq.knowledge.search(
    knowledge_id=kb_id,
    query="How do I reset my password?",
    retrieval_config={
        "type": "vector_search",
        "top_k": 3,
        "threshold": 0.7
    }
)
pretty_print(json.loads(search_results.model_dump_json()))

## 7. List all datasources

Endpoint: `GET /v2/knowledge/{knowledge_id}/datasources`

In [ ]:
kb_id = "" # enter the knowledge base id here

# List all datasources in a knowledge base (default limit of 10)
datasources = orq.knowledge.list_datasources(
    knowledge_id=kb_id,
    limit=10
)

if len(datasources.data) > 0:
   print(f"Datasources in knowledge base {kb_id}:")
   for datasource in datasources.data:
      print(datasource.id)
else:
   print(f"No datasources in knowledge base {kb_id}")

print("")
pretty_print(json.loads(datasources.model_dump_json()))

## 8. Create a new datasource

Endpoint: `POST /v2/knowledge/{knowledge_id}/datasources`

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here

# Create a pending datasource
datasource = orq.knowledge.create_datasource(
    knowledge_id=kb_id,
    display_name="Product Manual",
    chunking_options={
        "chunking_configuration": {
            "type": "advanced",
            "chunk_max_characters": 1000,
            "chunk_overlap": 100
        },
        "chunking_cleanup_options": {
            "clean_whitespaces": True,
            "clean_bullet_points": False,
            "clean_numbered_list": False
        }
    }
)

datasource_id = datasource.id
print(f"Created datasource with ID: {datasource_id}")
print("")
pretty_print(json.loads(datasource.model_dump_json()))

## 9. Retrieve a datasource

Endpoint: `GET /v2/knowledge/{knowledge_id}/datasources/{datasource_id}`

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here

retrieved_datasource = orq.knowledge.retrieve_datasource(
    knowledge_id = kb_id,  
    datasource_id = datasource_id
)

pretty_print(json.loads(retrieved_datasource.model_dump_json()))

## 10. Update a datasource

Endpoint: `PATCH /v2/knowledge/{knowledge_id}/datasources/{datasource_id}`

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here

# Example for `retrieveDatasource`
updated_datasource = orq.knowledge.update_datasource(
    knowledge_id = kb_id,  
    datasource_id = datasource_id,
    display_name = "updated_datasource"
)

pretty_print(json.loads(updated_datasource.model_dump_json()))



## 11. Deletes a datasource

Deletes a datasource from a knowledge base. Deleting a datasource will remove it from the knowledge base and all associated chunks. This action is irreversible and cannot be undone.

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here

# Delete a datasource
delete_datasource_result = orq.knowledge.delete_datasource(
    knowledge_id=kb_id,
    datasource_id=datasource_id
)
print(f"Datasource deleted successfully")

## 12. List all chunks for a datasource

Endpoint: `GET /v2/knowledge/{knowledge_id}/datasources/{datasource_id}/chunks`

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here

# List all chunks (default limit of 10)
chunk_list = orq.knowledge.list_chunks(
    knowledge_id = kb_id,    
    datasource_id= datasource_id
)

if len(chunk_list.data) > 0:
   print("Your chunks:")
   for chunk in chunk_list.data:
      print_chunk(chunk)
else:
   print("No chunks added yet")


## 13. Create chunks for a datasource

Endpoint: `POST /v2/knowledge/{knowledge_id}/datasources/{datasource_id}/chunks`

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here

# Create custom text chunks
chunks = orq.knowledge.create_chunks(
    knowledge_id=kb_id,
    datasource_id=datasource_id,
    request_body= [
        {
            "text": "To reset your password, navigate to the Settings page and click on 'Reset Password'. You will receive an email with further instructions.",
            "metadata": {
                "page_number": 12
            }
        },
        {
            "text": "The product supports integration with various third-party tools including Slack, Microsoft Teams, and Google Workspace.",
            "metadata": {
                "page_number": 15
            }
        }
    ]
)
for i, chunk in enumerate(chunks, start=1):
    print_chunk(chunk)


## 14. Retrieve a chunk

Endpoint: `GET /v2/knowledge/{knowledge_id}/datasources/{datasource_id}/chunks/{chunk_id}`

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here
chunk_id = "" # enter the chunk id here

# Example for `retireveChunk`
chunk = orq.knowledge.retrieve_chunk(
  knowledge_id = kb_id,    
  datasource_id= datasource_id,
  chunk_id = chunk_id,
)

print_chunk(chunk)

## 15. Update a chunk

Endpoint: `PATCH /v2/knowledge/{knowledge_id}/datasources/{datasource_id}/chunks/{chunk_id}`

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here
chunk_id = "" # enter the chunk id here

chunk = orq.knowledge.retrieve_chunk(
  knowledge_id = kb_id,    
  datasource_id= datasource_id,
  chunk_id = chunk_id,
)
print_chunk(chunk)

## 16. Delete a chunk

Endpoint: `DELETE /v2/knowledge/{knowledge_id}/datasources/{datasource_id}/chunks/{chunk_id}`

In [ ]:
kb_id = "" # enter the knowledge base id here
datasource_id = "" # enter the datasource id here
chunk_id = "" # enter the chunk id here

response = orq.knowledge.delete_chunk(
  knowledge_id = kb_id,    
  datasource_id= datasource_id,
  chunk_id = chunk_id,
)
print(response)